In [1]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "dataset/master.csv"
df = pd.read_csv(filepath)
df

,page_id,page_title,view_count,protection_status,page_length,edit_count,page_watchers,page_watchers_recent_edits,page_views_past_30days,page_id_scrapped,redirects,recent_number_of_edits,number_distinct_authors,number_categories
0,11291,Floccinaucinihilipilification,160037,autoconfirmed,373.0,709.0,69,8,484.0,11291.0,12.0,0.0,0.0,3.0
1,1143923,Honolulu University,41519,autoconfirmed,3365.0,165.0,Fewer than 30 watchers,NaN,461.0,1143923.0,1.0,0.0,0.0,3.0
2,1001251,Robert Spencer (author),532302,autoconfirmed,65.0,2.0,210,41,1825.0,60596940.0,0.0,0.0,0.0,3.0
3,1126490,Yard of ale,590917,autoconfirmed,5300.0,352.0,41,4,5641.0,1126490.0,6.0,0.0,0.0,3.0
4,1033826,Professional wrestling attacks,2967525,autoconfirmed,91.0,1.0,376,7,6476.0,62814991.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,18342008,"Carlos, Duke of Parma",241878,unprotected,59.0,1.0,Fewer than 30 watchers,NaN,2424.0,52782565.0,0.0,0.0,0.0,NaN
829,40792858,2013–14 MŠK Žilina season,3325,unprotected,4803.0,11.0,Fewer than 30 watchers,NaN,9.0,40792858.0,3.0,1.0,1.0,NaN
830,14531947,Politics of Anhui,44043,unprotected,3579.0,64.0,Fewer than 30 watchers,NaN,35.0,14531947.0,3.0,2.0,1.0,NaN
831,5482190,Dan Sikes,28302,unprotected,13221.0,123.0,Fewer than 30 watchers,NaN,182.0,5482190.0,0.0,1.0,1.0,NaN


In [2]:
features = df

# Labels are the values we want to predict
labels = np.array(df['protection_status'])

# Remove the labels from the features
drop_column_list = ['protection_status', 
                    'page_title', 
                    'page_id', 
                    'page_id_scrapped', 
                    'page_length', 
                    'page_watchers_recent_edits', 
                    'redirects', 
                    'recent_number_of_edits'
                   ]
features = features.drop(drop_column_list, axis =1)

features

,view_count,edit_count,page_watchers,page_views_past_30days,number_distinct_authors,number_categories
0,160037,709.0,69,484.0,0.0,3.0
1,41519,165.0,Fewer than 30 watchers,461.0,0.0,3.0
2,532302,2.0,210,1825.0,0.0,3.0
3,590917,352.0,41,5641.0,0.0,3.0
4,2967525,1.0,376,6476.0,0.0,3.0
...,...,...,...,...,...,...
828,241878,1.0,Fewer than 30 watchers,2424.0,0.0,NaN
829,3325,11.0,Fewer than 30 watchers,9.0,1.0,NaN
830,44043,64.0,Fewer than 30 watchers,35.0,1.0,NaN
831,28302,123.0,Fewer than 30 watchers,182.0,1.0,NaN


In [3]:
#Encode string to float
features.loc[features['page_watchers'] == "Fewer than 30 watchers", 'page_watchers'] = 25

#Convert String to Floats
features['edit_count'] = features['edit_count'].astype(float)
features['page_watchers'] = features['page_watchers'].astype(float)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)


In [5]:
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)   
        
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.20, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels

In [6]:
# kNN Imputer

from sklearn.impute import KNNImputer
imputer = KNNImputer (n_neighbors = 5)

X_train = imputer.fit_transform(X_train)

In [15]:
# Scaling 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [22]:
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_test, y_train, y_test)
# print(models)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

1.0

In [32]:
from sklearn.impute import KNNImputer
imputer = KNNImputer (n_neighbors = 5)
X_test = imputer.fit_transform(X_test)

In [33]:
X_test_std = scaler.transform(X_test)
clf.score(X_test_std, y_test)

0.5149700598802395

In [34]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)

In [44]:
features_pca = pca.fit(features)

features_pca = pca.transform(features)

In [38]:
# kNN Imputer

from sklearn.impute import KNNImputer
imputer = KNNImputer (n_neighbors = 5)

features = imputer.fit_transform(features)

In [40]:
print(pca.explained_variance_ratio_)

[9.99999995e-01 5.38590173e-09 9.79834904e-12 2.98077821e-14]


In [42]:
print(features_pca)

PCA(n_components=4)


In [46]:
features_pca.shape

(833, 4)

In [47]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.20, random_state = 25)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels

In [48]:
# Scaling 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [51]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, scaler.transform(X_test), y_train, y_test)
print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.61it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
XGBClassifier                      0.63               0.62    None      0.63   
ExtraTreesClassifier               0.62               0.61    None      0.62   
LGBMClassifier                     0.61               0.61    None      0.61   
RandomForestClassifier             0.61               0.60    None      0.61   
BaggingClassifier                  0.57               0.57    None      0.57   
KNeighborsClassifier               0.57               0.56    None      0.56   
LinearSVC                          0.57               0.56    None      0.53   
DecisionTreeClassifier             0.55               0.55    None      0.55   
LogisticRegression                 0.54               0.54    None      0.50   
NearestCentroid                    0.54               0.53    None      0.52   
PassiveAggressiveClassifier        0.54 